In [1]:
dataset_id = "college2_staging"

In [2]:
!bq --location=US mk --dataset {dataset_id}

Dataset 'sound-cider-252823:college2_staging' successfully created.


## Loading CSV files from Cloud Storage

In [5]:
!bq --location=US load --autodetect --skip_leading_rows=1 \
--source_format=CSV {dataset_id}.Current_Student \
'gs://college_dataset_utcs327e/rawData/Current_Students.csv'

Waiting on bqjob_r348b397c114d28bf_0000016e5d20aa45_1 ... (5s) Current status: DONE   


In [6]:
%%bigquery
select count(*) from college2_staging.Current_Student

,f0_
0,11


In [8]:
!bq --location=US load --autodetect --skip_leading_rows=1 \
--source_format=CSV {dataset_id}.New_Students \
'gs://college_dataset_utcs327e/rawData/New_Students.csv'

Waiting on bqjob_r3ddbad7e4e3e479_0000016e5d22b4a0_1 ... (2s) Current status: DONE   


In [9]:
%%bigquery
select count(*) from college2_staging.New_Students

,f0_
0,7


In [10]:
!bq --location=US load --autodetect --skip_leading_rows=1 \
--source_format=CSV {dataset_id}.Classes \
'gs://college_dataset_utcs327e/rawData/Classes.csv'

Waiting on bqjob_r7a5c27e6389f9d80_0000016e5d239660_1 ... (2s) Current status: DONE   


In [11]:
%%bigquery
select count(*) from college2_staging.Classes

,f0_
0,12


## Creating Modeled Tables

In [12]:
modeled_dataset_id = "college2_modeled"

In [13]:
!bq --location=US mk --dataset {modeled_dataset_id}

Dataset 'sound-cider-252823:college2_modeled' successfully created.


In [15]:
%%bigquery
create or replace table college2_modeled.Student as
select distinct sid, fname, lname, dob
from college2_staging.Current_Student
union distinct
select distinct sid, fname, lname, cast(dob as string) as dob
from college2_staging.New_Students

""


In [16]:
%%bigquery
select count(*) from college2_modeled.Student

,f0_
0,15


In [18]:
%%bigquery
create or replace table college2_modeled.Teacher as
select distinct tid, instructor, dept
from college2_staging.Classes

""


In [19]:
%%bigquery
select count(*) from college2_modeled.Teacher

,f0_
0,12


In [20]:
%%bigquery
select * from college2_modeled.Teacher

,tid,instructor,dept
0,cannata,PHILIP CANNATA,CS
1,None,Prof. Glenn Downing,CS
2,cannata,"Cannata, Philip",CS
3,None,Prof. Alison Norman,CS
4,mitra,"Mitra, Shyamal",CS
5,mueller,"MUELLER, PETER",Math
6,koch,"Koch, Hans",Math
7,tran,"Tran, Ngoc",Mathematics
8,neeman,JOE NEEMAN,Mathematics
9,mitra,"MITRA, SHYAMAL",Computer Science


In [22]:
%%bigquery
create or replace table college2_modeled.Takes as
select distinct sid, cno, grade
from college2_staging.Current_Student
where sid is not null
and cno is not null

""


In [23]:
%%bigquery
select * from college2_modeled.Takes

,sid,cno,grade
0,jc,CS327E,None
1,kev18,CS329E,None
2,paulg,CS329E,A
3,paulg,CS326E - Fall18,A-
4,paulg,CS313E,None
5,bzen26,CS313E - Fall18,B+
6,jerryh,CS327E,None
7,jerryh,CS327E,B
8,jerryh,CS329E,A-


In [24]:
%%bigquery
create table college2_modeled.Teaches as
select distinct tid, cno
from college2_staging.Classes
where tid is not null
and cno is not null

""


In [25]:
%%bigquery
select * from college2_modeled.Teaches

,tid,cno
0,koch,M328K
1,tran,M358K
2,bulko,CS313E
3,mitra,CS313E
4,mitra,CS329E
5,neeman,M362K
6,scohen,CS327E
7,cannata,CS347
8,cannata,CS326E
9,mueller,M362K
